In [2]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import math


columns = ['물건번호', '상가구분', '매매/전세/월세 구분', '가격', '계약면적(m2)', '전용면적(m2)', '보증금', '월세', '층수(물건층/전체층)', '기타 정보', '부동산', '상세정보 링크']
data = []

keyword = "파주시 문산읍"

url = "https://m.land.naver.com/search/result/{}".format(keyword)
res = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
res.raise_for_status()
soup = (str)(BeautifulSoup(res.text, "lxml"))

# print(soup)    # 파싱 확인용

In [3]:
#응답 메시지 속에서 원하는 데이터 얻기
#        filter: {
#            lat: '37.852847',
#            lon: '126.783632',
#            z: '14',
#            cortarNo: '4148025000',
#            cortarNm: '문산읍',
#           rletTpCds: '*',
#          tradTpCds: 'A1:B1:B2'
#     },

value = soup.split("filter: {")[1].split("}")[0].replace(" ","").replace("'","")

lat = value.split("lat:")[1].split(",")[0]
lon = value.split("lon:")[1].split(",")[0]
z = value.split("z:")[1].split(",")[0]
cortarNo = value.split("cortarNo:")[1].split(",")[0]
rletTpCds = value.split("rletTpCds:")[1].split(",")[0]
tradTpCds = value.split("tradTpCds:")[1].split()[0]

# lat - btm : 37.852847 - 37.8062736 = 0.046
# top - lat : 37.899391 - 37.852847 = 0.046
lat_margin = 0.046

# lon - lft : 126.783632 - 126.7297732 = 0.053
# rgt - lon : 126.8374908 - 126.783632 = 0.053
lon_margin = 0.053

btm=float(lat)-lat_margin
lft=float(lon)-lon_margin
top=float(lat)+lat_margin
rgt=float(lon)+lon_margin

# 최초 요청 시 디폴트 값으로 설정되어 있으나, 원하는 값으로 구성
rletTpCds="OPST" #오피스텔
tradTpCds="A1:B1:B2" #매매/전세/월세 매물 확인("A1:B1:B2")


In [4]:
# clusterList?view 를 통한 그룹(단지)의 데이터를 가져온다.
remaked_URL = "https://m.land.naver.com/cluster/clusterList?view=atcl&cortarNo={}&rletTpCd={}&tradTpCd={}&z={}&lat={}&lon={}&btm={}&lft={}&top={}&rgt={}"\
     .format(cortarNo, rletTpCds, tradTpCds, z, lat, lon,btm,lft,top,rgt)

res2 = requests.get(remaked_URL, headers={'User-agent': 'Mozilla/5.0'})
json_str = json.loads(json.dumps(res2.json()))


In [5]:
values = json_str['data']['ARTICLE']

# 큰 원으로 구성되어 있는 전체 매물그룹(values)을 load 하여 한 그룹씩 세부 쿼리 진행
for v in values:
    lgeo = v['lgeo']
    count = v['count']
    z2 = v['z']
    lat2 = v['lat']
    lon2 = v['lon']

    len_pages = count / 20 + 1
    for idx in range(1, math.ceil(len_pages)):
        
        remaked_URL2 = "https://m.land.naver.com/cluster/ajax/articleList?""itemId={}&mapKey=&lgeo={}&showR0=&" \
               "rletTpCd={}&tradTpCd={}&z={}&lat={}&""lon={}&totCnt={}&cortarNo={}&page={}"\
            .format(lgeo, lgeo, rletTpCds, tradTpCds, z2, lat2, lon2, count,cortarNo, idx)
        res3 = requests.get(remaked_URL2, headers={'User-agent': 'Mozilla/5.0'})
        json_str2 = json.loads(json.dumps(res3.json()))
        a = json_str2['body']
        for b in a:
            atclNo = b['atclNo']        # 물건번호
            rletTpNm = b['rletTpNm']    # 상가구분
            tradTpNm = b['tradTpNm']    # 매매/전세/월세 구분
            prc = b['prc']              # 가격
            spc1 = b['spc1']            # 계약면적(m2) -> 평으로 계산 : * 0.3025
            spc2 = b['spc2']            # 전용면적(m2) -> 평으로 계산 : * 0.3025
            hanPrc = b['hanPrc']        # 보증금                
            rentPrc = b['rentPrc']      # 월세
            flrInfo = b['flrInfo']      # 층수(물건층/전체층)
            tagList = b['tagList']      # 기타 정보
            rltrNm = b['rltrNm']        # 부동산
            detaild_information = "https://m.land.naver.com/article/info/{}".format(atclNo)
            data.append([atclNo, rletTpNm, tradTpNm, prc, spc1, spc2, hanPrc, rentPrc, flrInfo, tagList, rltrNm, detaild_information])
            
df = pd.DataFrame(data, columns=columns)

In [10]:
pd.options.display.max_rows = None  
df.sort_values(by=['월세', '계약면적(m2)'], ascending=[True, False], inplace=True)
df2 = df[df['매매/전세/월세 구분'] == '월세']

In [11]:
#df2.to_excel('/Users/kwon/Desktop/부동산 분석/파주시 문산읍(5월 2일기준 오피스텔).xlsx', )
df.to_excel(r"C:\Users\Kwon\OneDrive\바탕 화면\House\파주시 문산읍.xlsx", )
df2

,물건번호,상가구분,매매/전세/월세 구분,가격,계약면적(m2),전용면적(m2),보증금,월세,층수(물건층/전체층),기타 정보,부동산,상세정보 링크
9,2348184013,오피스텔,월세,300,74,33.12,300,30,6/7,"[15년이내, 대형평수, 화장실한개]",오천사부동산공인중개사사무소,https://m.land.naver.com/article/info/2348184013
8,2346611031,오피스텔,월세,500,31,18.59,500,30,4/4,"[15년이내, 화장실한개, 소형평수]",주공공인,https://m.land.naver.com/article/info/2346611031
11,2345099068,오피스텔,월세,300,74,33.12,300,35,6/7,"[15년이내, 융자금없는, 테라스]",리치부동산 공인중개사사무소,https://m.land.naver.com/article/info/2345099068
13,2348900865,오피스텔,월세,300,74,33.12,300,35,6/7,"[15년이내, 화장실한개, 방한개, 고층]",리치부동산공인중개사사무소,https://m.land.naver.com/article/info/2348900865
14,2348456914,오피스텔,월세,300,74,33.12,300,35,7/7,"[15년이내, 대형평수, 탑층, 화장실한개]",오천사부동산공인중개사사무소,https://m.land.naver.com/article/info/2348456914
10,2349007301,오피스텔,월세,300,74,33.12,300,38,3/7,"[15년이내, 융자금없는, 테라스]",리치부동산 공인중개사사무소,https://m.land.naver.com/article/info/2349007301
7,2347314821,오피스텔,월세,1000,75,26.8,"1,000",60,6/7,"[4년이내, 방한개, 화장실한개, 고층]",현대힐스테이트공인중개사무소,https://m.land.naver.com/article/info/2347314821
20,2345609977,오피스텔,월세,1000,106,74.78,"1,000",60,고/14,"[10년이내, 방세개, 화장실두개, 고층]",파주비젼부동산공인중개사사무소,https://m.land.naver.com/article/info/2345609977
6,2347595959,오피스텔,월세,500,104,37.34,500,65,7/7,"[4년이내, 탑층, 화장실한개, 방한개]",오천사부동산공인중개사사무소,https://m.land.naver.com/article/info/2347595959
17,2345095778,오피스텔,월세,1000,105,73.95,"1,000",70,4/14,"[10년이내, 화장실두개, 방세개]",리치부동산 공인중개사사무소,https://m.land.naver.com/article/info/2345095778
